In [ ]:
#Notes to self: 
#Median NDRE images have been created where one chunk == month
    #Should I then make an Image Collection from median images? Probably easier to export.
#Current code super bulky/slow. Looking for ways where code divides months for me.
    #filter.And()? Would that work?
#Image count seems to work, but it's giving me a giant number... What is it really counting??
#Dr. Wu has a time series function but not for sentinel.
#Matplotlib is super easy to use once I can export some dang LISTS
#Jan, Feb 2016 are missing data; March is basically just clouds

In [ ]:
#TODO:
#I need to find code that will let me export NDRE values to a list
    #So I can make a time series in matplotlib
    #Also system:time_start isn't working for me
    #Check code editor image count against python image count; maybe that will guide me
#Write cloud mask QA60 < 1; I have example from internet below
    #Is it really doing each individual layer?
        #Dunno cuz I can't COUNT how many images there are. Annoying.
#Look again @ Dr. Wu's code for exporting stats; maybe there's something I can use

In [1]:
import ee
import geemap
import os
import matplotlib as plt

In [2]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)

#Create a point for EC Flux Tower
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)

#Create a 0.5x0.5 deg region centered over the EC Flux Tower
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)

Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [3]:
#Add the polygon to the map
Map.addLayer(brma2_region, {}, 'Manauas Region')

In [4]:
#Rated Cloudiness on the following scale:
#Extra Cloudy > Very Cloudy > Fairly Cloudy > Fairly Clear > Very Clear > Extra Clear

In [5]:
#Formula for NDRE
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

#Set visual paramters for NDRE
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

In [6]:
#Create cloud mask; hasn't been implemented yet; will make a different one
def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

In [10]:
#Sentinel-2 January 2016
#Missing swath over study area
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-01-01', '2016-01-31')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- January 2016")

#Count something from the data; I don't think it's the number of images though.
count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_jan = sentinel2_ndre.median()
Map.addLayer(sentinel2_jan, ndre_params, 'Sentinel Median NDRE -- January 2016')

126804


In [11]:
#Sentinel-2 February 2016
#Missing swath over study area
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-02-01', '2016-02-29')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- February 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_feb = sentinel2_ndre.median()
Map.addLayer(sentinel2_feb, ndre_params, 'Sentinel Median NDRE -- February 2016')

99890


In [12]:
#Sentinel-2 March 2016
#Extra cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-03-01', '2016-03-31')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- March 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_mar = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_ndre_median = sentinel2_ndre.median()
Map.addLayer(sentinel2_mar, ndre_params, 'Sentinel Median NDRE -- March 2016')

159035


In [255]:
#Sentinel-2 April 2016
#Fairly cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-04-01', '2016-04-30')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- April 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_apr = sentinel2_ndre.median()
Map.addLayer(sentinel2_apr, ndre_params, 'Sentinel Median NDRE -- April 2016')

204384


In [256]:
#Sentinel-2 May 2016
#Very cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-05-01', '2016-05-31')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- May 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_may = sentinel2_ndre.median()
Map.addLayer(sentinel2_may, ndre_params, 'Sentinel Median NDRE -- May 2016')

233351


In [257]:
#Sentinel-2 June 2016
#Fairly cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-06-01', '2016-06-30')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- June 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_jun = sentinel2_ndre.median()
Map.addLayer(sentinel2_jun, ndre_params, 'Sentinel Median NDRE -- June 2016')

229380


In [258]:
#Sentinel-2 July 2016
#Very clear
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-07-01', '2016-07-31')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- July 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_jul = sentinel2_ndre.median()
Map.addLayer(sentinel2_jul, ndre_params, 'Sentinel Median NDRE -- July 2016')

116628


In [259]:
#Sentinel-2 August 2016
#Very clear
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-08-01', '2016-08-31')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- August 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_aug = sentinel2_ndre.median()
Map.addLayer(sentinel2_aug, ndre_params, 'Sentinel Median NDRE -- August 2016')

172711


In [260]:
#Sentinel-2 September 2016
#Very cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-09-01', '2016-09-30')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- September 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_sept = sentinel2_ndre.median()
Map.addLayer(sentinel2_sept, ndre_params, 'Sentinel Median NDRE -- September 2016')

224114


In [261]:
#Sentinel-2 October 2016
#Fairly cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-10-01', '2016-10-31')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- October 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_oct = sentinel2_ndre.median()
Map.addLayer(sentinel2_oct, ndre_params, 'Sentinel Median NDRE -- October 2016')

231216


In [262]:
#Sentinel-2 November 2016
#Very cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-11-01', '2016-11-30')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- November 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_nov = sentinel2_ndre.median()
Map.addLayer(sentinel2_nov, ndre_params, 'Sentinel Median NDRE -- November 2016')

219265


In [263]:
#Sentinel-2 December 2016
#Very/Extra Cloudy
sentinel2 = (ee.ImageCollection('COPERNICUS/S2') 
    .filterDate('2016-12-01', '2016-12-31')
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) 
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .map(lambda image : image.clip(brma2_region)))

#Show raw Sentinel imagery
vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 2000}
#Map.addLayer(sentinel2, vis_params, "Raw Sentinel -- December 2016")

count = sentinel2.size().getInfo()
print(count)

#Use NDRE function on all images and extract the NDRE band
sentinel2_ndre = sentinel2.map(addNDRE).select('NDRE')

#Calculate the median NDRE and add to the map
sentinel2_dec = sentinel2_ndre.median()
Map.addLayer(sentinel2_dec, ndre_params, 'Sentinel Median NDRE -- December 2016')

125482


In [264]:
#Put the EC Tower on top of previous layer
Map.addLayer(brma2_point, {}, 'Manauas EC Flux Location')

In [64]:
#Exported for good fun; way to set palette for export? Directory is hard coded.
out_dir = '/Users/morgansteckler/Desktop/'
filename = os.path.join(out_dir, 'sentinel_ndre.tif')
geemap.ee_export_image(median, filename = filename, scale = 90, region = square, file_per_band=False)

Generating URL ...
An error occurred while downloading.
'<' not supported between instances of 'builtin_function_or_method' and 'builtin_function_or_method'
